In [35]:
import json
import pprint

In [36]:
examples = ["""Dossier 72; Dossier 218 (Ms. "Impressive Farblehre", S. 1-19; S. 22-23) / Entität: Schwedischer Industrieller Typ: Person Rolle: Erwähnt (Empty) Bemerkungen: Person in einer von Donald Brinkmann #GND118674161 berichteten Anekdote. Der schwedische Industrielle gibt ein Nachtessen und führt in einem Versuch die expressive Wirkung von Farbe auf das Gemüt vor, indem er das Essen seiner Gäste in unterschiedlichen Farben beleuchten lässt (Quelle: Fd 13); Entität: Gäste eines schwedischen Industriellen Typ: Person Rolle: Erwähnt (Empty) Bemerkungen: Personen in einer von Donald Brinkmann #GND118674161 berichteten Anekdote. Die Gäste nehmen bei einem Nachtessen unfreiwillig an einem Versuch mit farbigem Licht zur Vorführung der expressiven Wirkung von Farbe auf das Gemüt teil (Quelle: Fd 13); Entität: südfranzösische Grossmutter Typ: Person Rolle: Erwähnt (Empty) Bemerkungen: Aus Südfrankreich stammende Grossmutter einer Schülerin oder eines Schülers aus Ittens #GND118710990 Klasse (Quelle: Fd 13); Entität: schottische Grossmutter Typ: Person Rolle: Erwähnt (Empty) Bemerkungen: Aus Schottland stammende Grossmutter einer Schülerin oder eines Schülers aus Ittens #GND118710990 Klasse (Quelle: Fd 13); Entität: mongolischer Grosselternteil Typ: Person Rolle: Erwähnt (Empty) Bemerkungen: Aus der Mongolei stammende Grossmutter oder Grossvater Nataschas, einer Schülerin aus Ittens #GND118710990 Klasse (Quelle: Fd 13); Entität: Natascha Typ: Person Rolle: Erwähnt (Empty) Bemerkungen: Schülerin Ittens #GND118710990, in Moskau #GND4074987-3 geboren, hat einen mongolischen Grossvater oder eine mongolische Grossmutter, vermutlich identisch mit Natascha D. s. HS NL 11: Fd 11 (Quelle: Fd 13); Entität: mongolischer Grosselternteil Nataschas Typ: Person Rolle: Erwähnt (Empty) Bemerkungen: Aus der Mongolei stammende Grossmutter oder Grossvater Nataschas, einer Schülerin aus Ittens #GND118710990 Klasse (Quelle: Fd 13)"""]

In [51]:
import re

class Parser:
    """
    Parser to extract structured information from free text remarks in the CMI export. 
    These remarks are currently used to capture information that cannot be stored in the reference fields. 
    For example, persons that cannot unambiguously be identified in the reference fields (e.g. "Schülerin Ittens", "BesucherInnen"),
    or entities for which no reference fields exist (events, works, etc.).

    The parser defines a standard set of fields and syntax for separating fields and records. However, these can be overrwriten by the user.

    Basic Usage:
    
    >>> p = Parser()
    >>> p.parse("Person: Elfriede Rolle: Erwähnt Bemerkungen: Eine Person namens Elfriede wurde erwähnt.")
    [{'Person': {'value': 'Elfriede'}, 'Rolle': {'value': 'Erwähnt'}, 'Bemerkungen': {'value': 'Eine Person namens Elfriede wurde erwähnt.'}}]

    Custom field and record separators:

    >>> p = Parser(fieldSeparator="::", recordSeparator="|")
    >>> p.parse("Person:: Olle Rolle:: Erwähnt | Person:: Ikke Rolle:: Erwähnt")
    [{'Person': {'value': 'Olle'}, 'Rolle': {'value': 'Erwähnt'}}, {'Person': {'value': 'Ikke'}, 'Rolle': {'value': 'Erwähnt'}}]

    Custom field definitions:

    >>> fields = {"type": {}, "identifier": {"options": {"qualifier": True}}}
    >>> p = Parser(fields=fields)
    >>> p.parse("type: place identifier: Q90 (Wikidata); type: place identifier: 4044660-8 (GND)")
    [{'type': {'value': 'place'}, 'identifier': {'value': 'Q90', 'qualifier': 'Wikidata'}}, {'type': {'value': 'place'}, 'identifier': {'value': '4044660-8', 'qualifier': 'GND'}}]
    """

    FIELDS = {
        "Anzahl": {
            "options": {
                "qualifier": True
            }
        },
        "Bemerkungen": {},
        "Entität": {
            "options": {
                "qualifier": True
            }
        },
        "Typ": {},
        "Person": {
            "options": {
                "qualifier": True
            }
        },
        "Rolle": {
            "options": {
                "qualifier": True
            }
        }
    }
    
    FIELD_SEPARATOR = ":"
    RECORD_SEPARATOR = ";"

    def __init__(self, *, fields=None, fieldSeparator=None, recordSeparator=None):
        if fields:
            self.FIELDS = fields
        if fieldSeparator:
            self.FIELD_SEPARATOR = fieldSeparator
        if recordSeparator:
            self.RECORD_SEPARATOR = recordSeparator
    
    def _extractRecordBlocks(self, text):
        try:
            records = [d.strip() for d in text.split(self.RECORD_SEPARATOR)]
        except Exception as e:
            raise e
        return records    
    
    def _parseRecordBlock(self, text):
        record = {}
        pattern = f'({ "|".join(self.FIELDS.keys())}){ self.FIELD_SEPARATOR}'
        matches = re.finditer(pattern, text)
        spans = []
        for match in matches:
            spans.append(match.span())
        for i, span in enumerate(spans):
            key = text[span[0]:span[1] - len(self.FIELD_SEPARATOR)]
            raw = text[span[1] + 1 : spans[i + 1][0] if i < len(spans) - 1 else None ].strip()
            if 'options' in self.FIELDS[key] and self.FIELDS[key]['options']['qualifier']:
                qualifier = re.search(r'\((.*?)\)$', raw)
                if qualifier:
                    value = raw.replace(f'({qualifier.group(1)})', '').strip()
                    qualifier = qualifier.group(1)
                    record = self._updateRecord(record, key=key, value=value, qualifier=qualifier)
                else:
                    record = self._updateRecord(record, key=key, value=raw)
            else:
                    record = self._updateRecord(record, key=key, value=raw)
        return record

    def _processIdentifiers(self, value):
        """
        If an identifier is set in the value (e.g. #GND4127793-4) extract them.
        The extracted identifier is then removed from the value.
        The function returns the changed value and a list of extracted identifiers
        """
        sources = ['GND']
        extractedIdentifiers = re.findall(r'#([\w\d\-]+)', value)
        if len(extractedIdentifiers):
            identifiers = []
            for extractedIdentifier in extractedIdentifiers:
                position = value.find("#%s" % extractedIdentifier)
                value = value.replace(f' #{extractedIdentifier}', '').strip()
                identifierObject = {'position': position}
                for source in sources:
                    if extractedIdentifier.startswith(source):
                        identifierObject['source'] = source
                        identifierObject['value'] = extractedIdentifier.replace(source, '')
                identifiers.append(identifierObject)
        return value, identifiers
        
    def _updateRecord(self, record, *, key, value, qualifier=False):
        obj = { 'value': value }
        if "#" in value:
            value, identifiers = self._processIdentifiers(value)
            obj['value'] = value
            obj['identifiers'] = identifiers
            
        if qualifier:
            obj['qualifier'] = qualifier
            
        if not key in record:
            # If key is not yet set we just add the data under the key
            record[key] = obj
        elif isinstance(record[key], list):
            # If the key already contains a list we add a new object to that list
            record[key].append(obj)
        else:
            # If the key exists but is not yet a list we convert it into a list, appendig the new object
            record[key] = [ record[key], obj]
        return record
    
    def parse(self, text):
        """
        Parse an internal remarks string into a set of records

        >>> p = Parser()
        >>> example1 = "Person: Hans Rolle: Erwähnt Bemerkungen: Eine Person namens Hans wurde erwähnt."
        >>> p.parse(example1)
        [{'Person': {'value': 'Hans'}, 'Rolle': {'value': 'Erwähnt'}, 'Bemerkungen': {'value': 'Eine Person namens Hans wurde erwähnt.'}}]

        >>> example2 = "Person: Schülerin Ittens Rolle: Erwähnt (Empty) Bemerkungen: Eine Schülerin, eventuell Natascha D., wird erwähnt."
        >>> p.parse(example2)
        [{'Person': {'value': 'Schülerin Ittens'}, 'Rolle': {'value': 'Erwähnt', 'qualifier': 'Empty'}, 'Bemerkungen': {'value': 'Eine Schülerin, eventuell Natascha D., wird erwähnt.'}}]

        >>> example3 = "Person: BesucherInnen Rolle: Erwähnt Anzahl: >1; Person: Mitarbeitende Rolle: Erwähnt Anzahl: 10 Bemerkungen: Die Mitarbeitenden haben die Ausstellung betreut"
        >>> p.parse(example3)
        [{'Person': {'value': 'BesucherInnen'}, 'Rolle': {'value': 'Erwähnt'}, 'Anzahl': {'value': '>1'}}, {'Person': {'value': 'Mitarbeitende'}, 'Rolle': {'value': 'Erwähnt'}, 'Anzahl': {'value': '10'}, 'Bemerkungen': {'value': 'Die Mitarbeitenden haben die Ausstellung betreut'}}]

        >>> example4 = 'Person: Direktor (Chirurgische Klinik (Zürich)) Rolle: Erwähnt (Empty) Bemerkungen: vermutlich Brunner, Alfred (Link zur GND: <a href=http://d-nb.info/gnd/13133056Xtarget="_blank">GND</a>)'
        >>> p.parse(example4)
        [{'Person': {'value': 'Direktor', 'qualifier': 'Chirurgische Klinik (Zürich)'}, 'Rolle': {'value': 'Erwähnt', 'qualifier': 'Empty'}, 'Bemerkungen': {'value': 'vermutlich Brunner, Alfred (Link zur GND: <a href=http://d-nb.info/gnd/13133056Xtarget="_blank">GND</a>)'}}]

        >>> example5 = 'Entität: Mona Lisa #GND4074156-4 Typ: Werk Rolle: Erwähnt Bemerkung: ...'
        >>> p.parse(example5)
        [{'Entität': {'value': 'Mona Lisa #GND4074156-4'}, 'Typ': {'value': 'Werk'}, 'Rolle': {'value': 'Erwähnt Bemerkung: ...'}}]
        """
        records = []
        recordBlocks = self._extractRecordBlocks(text)
        for recordBlock in recordBlocks:
            parsedBlock = self._parseRecordBlock(recordBlock)
            if len(parsedBlock):
                records.append(parsedBlock)
        return records


p = Parser()

In [52]:
multipleRoleExample = """Entität: Neue Person Rolle: Erwähnt (Empty) Rolle: Abgebildet Bemerkungen: Die Person wird erwähnt und skizziert
"""
r = p.parse(multipleRoleExample)
print(json.dumps(r, indent=4))

[
    {
        "Entit\u00e4t": {
            "value": "Neue Person"
        },
        "Rolle": [
            {
                "value": "Erw\u00e4hnt",
                "qualifier": "Empty"
            },
            {
                "value": "Abgebildet"
            }
        ],
        "Bemerkungen": {
            "value": "Die Person wird erw\u00e4hnt und skizziert"
        }
    }
]


In [53]:
r = p.parse(examples[0])
print(json.dumps(r, indent=4))

[
    {
        "Entit\u00e4t": {
            "value": "Schwedischer Industrieller"
        },
        "Typ": {
            "value": "Person"
        },
        "Rolle": {
            "value": "Erw\u00e4hnt",
            "qualifier": "Empty"
        },
        "Bemerkungen": {
            "value": "Person in einer von Donald Brinkmann berichteten Anekdote. Der schwedische Industrielle gibt ein Nachtessen und f\u00fchrt in einem Versuch die expressive Wirkung von Farbe auf das Gem\u00fct vor, indem er das Essen seiner G\u00e4ste in unterschiedlichen Farben beleuchten l\u00e4sst (Quelle: Fd 13)",
            "identifiers": [
                {
                    "position": 37,
                    "source": "GND",
                    "value": "118674161"
                }
            ]
        }
    },
    {
        "Entit\u00e4t": {
            "value": "G\u00e4ste eines schwedischen Industriellen"
        },
        "Typ": {
            "value": "Person"
        },
        "Rolle": {
 